# Multilabel BERT Experiments

In this notebook we do some first experiments with BERT: we finetune a BERT model+classifier on each of our datasets separately and compute the accuracy of the resulting classifier on the test data.

For these experiments we use the `pytorch_transformers` package. It contains a variety of neural network architectures for transfer learning and pretrained models, including BERT and XLNET.

Two different BERT models are relevant for our experiments: 

- BERT-base-uncased: a relatively small BERT model that should already give reasonable results,
- BERT-large-uncased: a larger model for real state-of-the-art results.

In [1]:
PREFIXES = ["voting_but", "voting_so", "voting_because"]  #"voting_so_automl"
BERT_MODEL = 'bert-base-uncased'
BATCH_SIZE = 16 if "base" in BERT_MODEL else 2
GRADIENT_ACCUMULATION_STEPS = 1 if "base" in BERT_MODEL else 8
MAX_SEQ_LENGTH = 100

## Data

We use the same data as for all our previous experiments. Here we load the training, development and test data for a particular prompt.

In [2]:
import sys
sys.path.append('../')

import ndjson
import glob
import numpy as np
from collections import Counter

from quillnlp.models.bert.preprocessing import preprocess, create_label_vocabulary

data = []
for prefix in PREFIXES:
    data_file = f"../data/interim/{prefix}_withprompt.ndjson"

    with open(data_file) as i:
        data += ndjson.load(i)

label2idx = create_label_vocabulary(data)
idx2label = {v:k for k,v in label2idx.items()}
target_names = [idx2label[s] for s in range(len(idx2label))]
data_items = np.array(preprocess(data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH))

labels = Counter()
for item in data:
    labels.update(item["label"])
        
print(labels)

I0327 11:45:22.739525 139635286574912 file_utils.py:41] PyTorch version 1.2.0+cu92 available.
I0327 11:45:23.721752 139635286574912 file_utils.py:57] TensorFlow version 2.1.0 available.
I0327 11:45:24.354614 139635286574912 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


Counter({'Personal Opinion to Fact': 617, 'Increases Voter Turnout': 427, 'Rep': 371, 'Increases Representation': 239, 'Vote': 218, 'Other': 197, 'Leaders': 109, 'Otherwise Voter Turnout will Decrease/Will be Low': 93, "Outside of Article's Scope": 74, 'Governments Can Abuse It/Dictatorships': 52, 'Apathy': 40, 'Ensures a Representative Democracy': 35, "Outside of article's scope": 35, 'Unintelligible Response': 25, 'Duty': 25, 'Unintelligible response': 25, 'Answered wrong stem': 22, 'Negative Effects on Voter': 20, 'May Cause Low Voter Turnout': 20, 'How--Mechanism': 19, 'Misuse of Conjunction': 16, 'Answered Wrong Stem': 15, 'Violation of Personal Liberty': 14, 'Reference': 14, 'Cause/Effect Misattribution': 14, 'Plagiarism': 13, 'Not All Governments Enforce It': 10, 'North Korea/Dictatorships': 10, 'Consequences/Bolivia': 10, 'Clarify/Identify/Specify': 10, 'Evidences Needs More Context': 10, 'Feel': 10, 'Otherwise Not Representative': 10, 'Not Representative': 10, 'Otherwise Voter

Read synthetic data, when available.

## Model

We load the pretrained model and put it on a GPU if one is available. We also put the model in "training" mode, so that we can correctly update its internal parameters on the basis of our data sets.

In [3]:
def evaluate_output(all_correct, all_predicted):
    correct = 0
    at_least_one = 0
    fp, fn, tp, tn = 0, 0, 0, 0
    for c, p in zip(all_correct, all_predicted):
        if sum(c == p) == len(c):
            correct +=1

        for ci, pi in zip(c, p):
            if pi == 1 and ci == 1:
                at_least_one += 1
                break

        for ci, pi in zip(c, p):
            if pi == 1 and ci == 1:
                tp += 1
                same = 1
            elif pi == 1 and ci == 0:
                fp += 1
            elif pi == 0 and ci == 1:
                fn += 1
            else:
                tn += 1
                same =1

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    print("Data size:", len(all_predicted))
    print("P:", tp, "/", tp+fp, "=", precision)
    print("R:", tp, "/", tp+fn, "=", recall)
    print("F:", 2*precision*recall/(precision+recall))
    print("A:", correct/len(all_correct))
    print("AL1:", at_least_one/len(all_correct))

In [4]:
import torch

from quillnlp.models.bert.train import train, evaluate
from quillnlp.models.bert.models import get_multilabel_bert_classifier

from quillnlp.models.bert.preprocessing import get_data_loader
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=1)
all_correct, all_predicted = [], []
all_test_data = []
for train_idx, test_idx in kf.split(data_items):

    train_and_dev_data = data_items[train_idx]
    cutoff = int(len(train_and_dev_data)/6*5)
    train_data = train_and_dev_data[:cutoff]
    dev_data = train_and_dev_data[cutoff:]
    test_data = data_items[test_idx]
    
    print("Train size:", len(train_data))
                    
    train_dataloader = get_data_loader(train_data, BATCH_SIZE)
    dev_dataloader = get_data_loader(dev_data, BATCH_SIZE)
    test_dataloader = get_data_loader(test_data, BATCH_SIZE, shuffle=False)

    print("Final train dataloader length:", len(train_dataloader))
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = get_multilabel_bert_classifier(BERT_MODEL, len(label2idx), device=device)
    output_model_file = train(model, train_dataloader, dev_dataloader, BATCH_SIZE, GRADIENT_ACCUMULATION_STEPS, device)
    
    print("Loading model from", output_model_file)
    device="cpu"

    model = get_multilabel_bert_classifier(BERT_MODEL, len(label2idx), model_file=output_model_file, device=device)
    model.eval()
    
    _, _, test_correct, test_predicted = evaluate(model, test_dataloader, device)
    evaluate_output(test_correct, test_predicted)
    all_correct.extend(test_correct)
    all_predicted.extend(test_predicted)
    all_test_data.extend(test_data)

Train size: 1608
Final train dataloader length: 101


I0327 11:45:25.643782 139635286574912 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0327 11:45:25.645409 139635286574912 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd


Loss history: []
Dev loss: 0.2995312355813526


Epoch:   5%|▌         | 1/20 [00:22<07:07, 22.52s/it]


Loss history: [0.2995312355813526]
Dev loss: 0.15137098942484176


...




Epoch:  80%|████████  | 16/20 [06:03<01:30, 22.52s/it]


Loss history: [0.30039810566675096, 0.14583712887196315, 0.11976408639124461, 0.10973445858274188, 0.1026154046966916, 0.09780198548521314, 0.09671932813667115, 0.0946484353570711, 0.09589916325750805, 0.09617072095473607, 0.09707399422214144, 0.09415223733300254, 0.09524037192265193, 0.09895849902005423, 0.09688178556306022]
Dev loss: 0.09899386018514633



Loss history: [0.30039810566675096, 0.14583712887196315, 0.11976408639124461, 0.10973445858274188, 0.1026154046966916, 0.09780198548521314, 0.09671932813667115, 0.0946484353570711, 0.09589916325750805, 0.09617072095473607, 0.09707399422214144, 0.09415223733300254, 0.09524037192265193, 0.09895849902005423, 0.09688178556306022, 0.09899386018514633]
Dev loss: 0.09674237952345893
No improvement on development set. Finish training.
Loading model from /tmp/model.bin


I0327 12:17:42.185212 139635286574912 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0327 12:17:42.186827 139635286574912 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd


Data size: 482
P: 276 / 345 = 0.8
R: 276 / 579 = 0.47668393782383417
F: 0.5974025974025974
A: 0.4149377593360996
AL1: 0.4854771784232365


## Evaluation

In [5]:
evaluate_output(all_correct, all_predicted)

Data size: 2413
P: 1288 / 1546 = 0.833117723156533
R: 1288 / 2907 = 0.4430684554523564
F: 0.5784864136537166
A: 0.39204309987567343
AL1: 0.449647741400746


In [6]:
scores = {}
for item, predicted, correct in zip(all_test_data, all_predicted, all_correct):
    correct_labels = [idx2label[i] for i, l in enumerate(correct) if l == 1]
    predicted_labels = [idx2label[i] for i, l in enumerate(predicted) if l == 1]
    print("{}#{}#{}".format(item.text, ";".join(correct_labels), ";".join(predicted_labels)))    
        
    for label in predicted_labels + correct_labels:
        if label not in scores:
            scores[label] = {"tp": 0, "fp": 0, "fn": 0, "support": 0}
    
    for label in predicted_labels:
        if label in correct_labels:
            scores[label]["tp"] += 1
        else:
            scores[label]["fp"] += 1

    for label in correct_labels:
        scores[label]["support"] += 1
        if label not in predicted_labels:
            scores[label]["fn"] += 1
            
for label in scores:
    lp = scores[label]["tp"] / (scores[label]["tp"] + scores[label]["fp"]) if scores[label]["tp"] + scores[label]["fp"] > 0 else 0
    lr = scores[label]["tp"] / (scores[label]["tp"] + scores[label]["fn"]) if scores[label]["tp"] + scores[label]["fn"] > 0 else 0
    lf = 2*lp*lr/(lp+lr) if lp+lr > 0 else 0
    
    print(label, lp, lr, lf, scores[label]["support"])

compulsory voting can often be misused by governments and many feel it is a violation of personal liberties and choices.#Governments Can Abuse It/Dictatorships#
it can be misused.#Governments Can Abuse It/Dictatorships#
, sometimes the government abuses compulsory voting.#Governments Can Abuse It/Dictatorships#
dictator-like governments can abuse it to support certain political groups.#Governments Can Abuse It/Dictatorships#
when misused by the government does not allow for competition.#Governments Can Abuse It/Dictatorships#Personal Opinion to Fact
this may cause hardships for voters who are not able to make it for good reasons#Negative Effects on Voter#
this may have adverse effects on voters#Negative Effects on Voter#
not all countries follow the rule.#Not All Governments Enforce It#
Not all countries with compulsory voting enforce this law#Not All Governments Enforce It#
not every country actually follows up with these laws#Not All Governments Enforce It#
governments can possibly m

make sure that voters have a way to get to the polls if they need help doing so.#Outside of Article's Scope#
need to make it MUCH easier to vote.#Outside of Article's Scope#Personal Opinion to Fact
pay people for their time, since it is required.#Outside of Article's Scope#Personal Opinion to Fact
the day upon which voting falls should be 24 hours in length to maximize people's options and time to vote.#Outside of Article's Scope#Personal Opinion to Fact
they could offer a small tax break to those who do.#Outside of Article's Scope#
they should give people an extended window of time in which to vote so they don't have to penalize people for not being able to vote on time.#Outside of Article's Scope#Personal Opinion to Fact
we also need to remove unlimited campaign contributions and better inform people of candidates' platforms.#Outside of Article's Scope#Personal Opinion to Fact
without flexibility on the timing for voting even though compulsory; persons might not be able to comply.#Ou

also offer more than one person to vote for on the ballot.#Personal Opinion to Fact#Personal Opinion to Fact
avoid punishing those who do not and have a good reason for doing so.#Personal Opinion to Fact#Personal Opinion to Fact
citizens should not be punished if they have a valid reason for not voting (i.e. in the hospital, no transportation, etc.).#Personal Opinion to Fact#Personal Opinion to Fact
citizens who can prove that they were unable to vote should not be punished#Personal Opinion to Fact#Personal Opinion to Fact
do not punish those that are unable to make it to the polls due to illness, disability, etc.#Personal Opinion to Fact#Personal Opinion to Fact
ensure laws are passed that don't allow for it to be misused.#Personal Opinion to Fact#Personal Opinion to Fact
establish laws that do not adversely punish people who are unable to vote for particular reasons.#Personal Opinion to Fact#Personal Opinion to Fact
give grounds for exemption & ask for a reason for exemption.#Persona

In [7]:
print(len([p for p in all_predicted if sum(p) == 0]), "/", len(all_predicted))

1154 / 2413
